In [1]:
from netCDF4 import Dataset
import numpy as np
from pathlib import Path
import os

In [2]:
dirpath = r"/home/mljc/wrf/out/idea-hill/"

savefilepath = "/home/mljc/wrf/out/ideal-hill/hill.pkl"


In [3]:
def get_array_from_fire_subgrid(var_name, root):
    sr_x=int(len(root.dimensions['west_east_subgrid'])/(len(root.dimensions['west_east'])+1))
    sr_y=int(len(root.dimensions['south_north_subgrid'])/(len(root.dimensions['south_north'])+1))
    data=root.variables[var_name]
    return np.array(data[:,:-sr_y,:-sr_x].data), sr_x, sr_y


def process_data(root, var, expr=None):
    arr = np.array(root.variables[var][:].data)    
    lats = root.variables['XLAT'][0, :, 0]
    longs = root.variables['XLONG'][0, 0, :]    
    if 'south_north_subgrid' in root.variables[var].dimensions:
        arr, sr_x, sr_y = get_array_from_fire_subgrid(var, root)
        longs = np.linspace(longs.min(), longs.max(), len(root.dimensions['west_east_subgrid'])-sr_x), 
        lats = np.linspace(lats.min(), lats.max(), len(root.dimensions['south_north_subgrid'])-sr_y)

    return arr, longs, lats

In [ ]:
with open(savefilepath, 'wb') as file:

    data = []
    for var in variables:
        i = 0
        save = True
        for filename in os.listdir(dirpath):
            if filename[:6] == "wrfout": 
                print(f"Processing {filename}: {var}")
                root = Dataset(os.path.join(dirpath, filename), "r", format="NETCDF4")
                try:
                    arr_temp, lon, lat = process_data(root, var)
                except Exception as e:
                    save = False
                    print(f"Could not save variable {var}")
                if i == 0 : 
                    arr_tot = arr_temp
                    i+=1
                else: 
                    np.append(arr_tot, arr_temp)
               
        if save:
            a = {
                "data" : arr_tot,
                "description": root.variables[var].description,
                "coordinates": root.variables[var].coordinates,
            }
            data.append(a)

    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)